In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
# View all columns
pd.set_option('display.max_columns', None)


In [2]:
cwd = Path.cwd()
cwd

PosixPath('/home/ivan/dev/git/slack_analytics')

### Step 0: Aggregate and Iterate over Slack Dump Zip File

In [3]:
# List Folders, which are the channels
data_dir = cwd / 'data'
subdirs = [d for d in data_dir.glob('*/') if d.is_dir()]

for subdir in subdirs:
    # print(subdir)
    pass

In [4]:
# Test on art channel directory
directory_path = cwd / 'data' / 'art'
rows = []

for json_file in directory_path.glob('*.json'):
    with open(json_file, 'r') as file:
        data = json.load(file)
        # Capture the parent directory name
        directory_name = json_file.parent.name
        if isinstance(data, list):  # If the JSON is a list
            for item in data:
                item['file_name'] = json_file.stem  # File name without extension
                item['directory_name'] = directory_name  # Directory name
                rows.append(item)
        else:  # If the JSON is a dictionary
            data['file_name'] = json_file.stem
            data['directory_name'] = directory_name
            rows.append(data)

# Normalize nested JSON data into a flat table
df = pd.json_normalize(rows)
df.head(3)

,user,type,ts,client_msg_id,text,team,user_team,source_team,attachments,blocks,file_name,directory_name,user_profile.avatar_hash,user_profile.image_72,user_profile.first_name,user_profile.real_name,user_profile.display_name,user_profile.team,user_profile.name,user_profile.is_restricted,user_profile.is_ultra_restricted,files,upload,display_as_bot,reactions,edited.user,edited.ts,thread_ts,parent_user_id,subtype,inviter,reply_count,reply_users_count,latest_reply,reply_users,replies,is_locked,subscribed,last_read,root.user,root.type,root.ts,root.client_msg_id,root.text,root.team,root.thread_ts,root.reply_count,root.reply_users_count,root.latest_reply,root.reply_users,root.replies,root.is_locked,root.subscribed,root.last_read,root.blocks,bot_id
0,UHHK3HG48,message,1702997743.251229,877ea291-54e4-4edd-aef3-c2774211fb58,<https://twitter.com/bugmeyer/status/173712326...,TH7UMG0KT,TH7UMG0KT,TH7UMG0KT,[{'from_url': 'https://twitter.com/bugmeyer/st...,"[{'type': 'rich_text', 'block_id': 'EUQVM', 'e...",2023-12-19,art,cc6f6744b587,https://avatars.slack-edge.com/2023-04-13/5105...,Michael,Michael,Michael,TH7UMG0KT,prohibited_art,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UHEB1D3PA,message,1703007892.274149,e6048fa7-cea9-4118-8287-e2bb561f5f3f,Yeah this guy does good shit,TH7UMG0KT,TH7UMG0KT,TH7UMG0KT,NaN,"[{'type': 'rich_text', 'block_id': '9Lp03', 'e...",2023-12-19,art,f77a8e01eea8,https://avatars.slack-edge.com/2023-04-17/5118...,Al,Al,Alejandro,TH7UMG0KT,alejandro.thornton,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UHH0R7TTP,message,1703008645.622239,27324906-7a9e-4ddf-83bf-f7557645f24c,wow so cool,TH7UMG0KT,TH7UMG0KT,TH7UMG0KT,NaN,"[{'type': 'rich_text', 'block_id': 'LKQmM', 'e...",2023-12-19,art,ac0243a91893,https://avatars.slack-edge.com/2020-01-28/9281...,Ivan,Ivan,,TH7UMG0KT,skantastico,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Now iterate over all channels
directory_path = cwd / 'data'
rows = []

# Iterate over each subdirectory in the directory_path
for subdir in directory_path.iterdir():
    if subdir.is_dir():  # Ensure it's a directory
        # Iterate over JSON files in the current subdirectory
        for json_file in subdir.glob('*.json'):
            with open(json_file, 'r') as file:
                data = json.load(file)
                # Use subdir.name to get the directory name
                if isinstance(data, list):
                    for item in data:
                        item['file_name'] = json_file.stem  # Add the file name
                        item['directory_name'] = subdir.name  # Add the directory name
                        rows.append(item)
                else:
                    data['file_name'] = json_file.stem
                    data['directory_name'] = subdir.name
                    rows.append(data)

# Normalize nested JSON data into a flat table
df = pd.json_normalize(rows)

In [6]:
# Save Raw File
save_dir = cwd / 'processed_json'

df.to_csv(f'{save_dir}/slack_snapshot_03_26_24.csv', index=False)

### Step 1: Get Relevant Columns and Pare it down

In [4]:
# Read from file instead of re-doing it over and over

cols_v1 = ['user_profile.real_name', 'directory_name',
           'ts','text', 'reply_count', 
           'reactions', 'replies', 'user_profile.name', 'user']
posts = pd.read_csv('processed_json/slack_snapshot_03_26_24.csv')
p2 = posts[cols_v1]
p2.shape

/tmp/ipykernel_16657/3637991842.py:6: DtypeWarning: Columns (37,38,42,43,45,46,47,48,49,50,51,53,54,55,56,58,59,60,61,64,65,66,67,68,69,70,71,82,83,85,86,87,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,109,110,112,113,114,115,116,117,119,120,121,122,124,125,127,128,129,130,131,132,133,134,135,136,137,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,163,164,165,166,167,168,169,170,171,172,173,174,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214) have mixed types. Specify dtype option on import or set low_memory=False.
  posts = pd.read_csv('processed_json/slack_snapshot_03_26_24.csv')


(844914, 9)

In [5]:
p2.head(5)

,user_profile.real_name,directory_name,ts,text,reply_count,reactions,replies,user_profile.name,user
0,Ivan,chess,1.663200e+09,<https://twitter.com/MetroUK/status/1569983251...,NaN,"[{'name': 'joy', 'users': ['UH81VL1TK', 'UHEB1...",NaN,skantastico,UHH0R7TTP
1,Ivan,chess,1.663200e+09,lol just a complete non story from the thread,NaN,NaN,NaN,skantastico,UHH0R7TTP
2,Ivan,chess,1.663200e+09,"But it's too late, this is the meme now",NaN,NaN,NaN,skantastico,UHH0R7TTP
3,Roth,chess,1.663200e+09,I think this is xQcs fault,NaN,NaN,NaN,rothgar2112,UHK1N52Q6
4,Al,chess,1.663213e+09,Lmao,NaN,NaN,NaN,alejandro.thornton,UHEB1D3PA


In [8]:
# convert ts to timestamp
p2['time'] = pd.to_datetime(p2['ts'], unit='s')
p2.drop('ts', axis=1, inplace=True)
# Rename and reorganize columns
p2 = p2.rename(columns={'user_profile.real_name':'user',
                        'directory_name':'channel',
                        'user_profile.name':'username',
                        'user':'user_id'})

/tmp/ipykernel_16657/3971780017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p2['time'] = pd.to_datetime(p2['ts'], unit='s')
/tmp/ipykernel_16657/3971780017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p2.drop('ts', axis=1, inplace=True)


In [11]:
p2.head(50)

NameError: name 'p2' is not defined

In [10]:
postdir = cwd / 'processed_json'
p2.to_csv(f'{postdir}/posts.csv', index=False)

### Step 2: Create Users and Add Data

In [6]:
userfile = 'data/users.json'
usercols = ['real_name', 'name', 'id', 'profile'] 
users = pd.read_json(userfile)
users
u2 = users.copy()[usercols]
u2

,real_name,name,id,profile
0,Stephen,stephen.leininger,UH81VL1TK,"{'title': '', 'phone': '', 'skype': '', 'real_..."
1,Derek,froglenzen,UHAC3TPCZ,"{'title': '', 'phone': '', 'skype': '', 'real_..."
2,Al,alejandro.thornton,UHEB1D3PA,"{'title': '', 'phone': '', 'skype': '', 'real_..."
3,Ivan,skantastico,UHH0R7TTP,"{'title': 'Praetor Exelcius', 'phone': '', 'sk..."
4,Will,william.west.lane,UHH7GF9S9,"{'title': '', 'phone': '', 'skype': '', 'real_..."
5,Michael,prohibited_art,UHHK3HG48,"{'title': '', 'phone': '', 'skype': '', 'real_..."
6,Ben,benskee33,UHHS66KU7,"{'title': '', 'phone': '', 'skype': '', 'real_..."
7,Roth,rothgar2112,UHK1N52Q6,"{'title': '', 'phone': '', 'skype': '', 'real_..."
8,Google Drive,googledrive,UKRTZJ649,"{'title': '', 'phone': '', 'skype': '', 'real_..."
9,scryfall,scryfall,UN85LHRNK,"{'title': '', 'phone': '', 'skype': '', 'real_..."


In [5]:
u2.to_csv('processed_json/users.csv', index=False)

In [9]:
# Get Postcount for user
posts = pd.read_csv('processed_json/posts.csv')
postcount = posts.groupby(['user', 'text']).size().reset_index(name='postcount')
userposts = postcount.groupby('user')['postcount'].sum().reset_index()
userposts = userposts.sort_values(by='postcount', ascending=False)
userposts

,user,postcount
3,Ivan,219761
4,Michael,142846
6,Stephen,119069
0,Al,108511
5,Roth,78886
7,Will,53571
2,Derek,45402
1,Ben,40286


In [10]:
# Alternate: Use user_id
postcount = posts.groupby(['user_id', 'text']).size().reset_index(name='postcount')
userposts = postcount.groupby('user_id')['postcount'].sum().reset_index()
userposts = userposts.sort_values(by='postcount', ascending=False)
userposts

,user_id,postcount
9,UHH0R7TTP,221865
11,UHHK3HG48,143292
6,UH81VL1TK,122867
8,UHEB1D3PA,110776
13,UHK1N52Q6,79395
10,UHH7GF9S9,54150
7,UHAC3TPCZ,46662
12,UHHS66KU7,41064
3,U04UVP3GJ5R,402
1,U01JFFEK6S2,64


In [13]:
# merge onto users
u3 = pd.merge(u2, userposts, left_on='id', right_on='user_id', how='left')
u3

,real_name,name,id,profile,user_id,postcount
0,Stephen,stephen.leininger,UH81VL1TK,"{'title': '', 'phone': '', 'skype': '', 'real_...",UH81VL1TK,122867.0
1,Derek,froglenzen,UHAC3TPCZ,"{'title': '', 'phone': '', 'skype': '', 'real_...",UHAC3TPCZ,46662.0
2,Al,alejandro.thornton,UHEB1D3PA,"{'title': '', 'phone': '', 'skype': '', 'real_...",UHEB1D3PA,110776.0
3,Ivan,skantastico,UHH0R7TTP,"{'title': 'Praetor Exelcius', 'phone': '', 'sk...",UHH0R7TTP,221865.0
4,Will,william.west.lane,UHH7GF9S9,"{'title': '', 'phone': '', 'skype': '', 'real_...",UHH7GF9S9,54150.0
5,Michael,prohibited_art,UHHK3HG48,"{'title': '', 'phone': '', 'skype': '', 'real_...",UHHK3HG48,143292.0
6,Ben,benskee33,UHHS66KU7,"{'title': '', 'phone': '', 'skype': '', 'real_...",UHHS66KU7,41064.0
7,Roth,rothgar2112,UHK1N52Q6,"{'title': '', 'phone': '', 'skype': '', 'real_...",UHK1N52Q6,79395.0
8,Google Drive,googledrive,UKRTZJ649,"{'title': '', 'phone': '', 'skype': '', 'real_...",UKRTZJ649,1.0
9,scryfall,scryfall,UN85LHRNK,"{'title': '', 'phone': '', 'skype': '', 'real_...",UN85LHRNK,10.0


### Step 3: Create Channels and Add Data

In [16]:
channelfile = 'data/channels.json'
channels = pd.read_json(channelfile)
channel_cols = ['name', 'id', 'creator', 'members', 'purpose', 'pins', 'topic', 'is_archived', 'is_general']
chans = channels[channel_cols]
chans.head(4)

,name,id,creator,members,purpose,pins,topic,is_archived,is_general
0,bookstream,CHBC77PMX,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'discussions and channeling of thoug...,"[{'id': '1555531733.250500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False
1,stackingsats,CHG467KSM,UHH0R7TTP,"[UH81VL1TK, UHEB1D3PA, UHH0R7TTP, UHH7GF9S9, U...","{'value': 'Stacking sats', 'creator': 'UHH0R7T...","[{'id': '1555032808.005500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False
2,annoucements,CHGKGAJJY,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'This channel is for workspace-wide ...,"[{'id': '1554065897.002200', 'type': 'C', 'cre...","{'value': 'Schedule and Info', 'creator': 'UHH...",False,True
3,shitposting,CHHPRTHGF,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1555442461.358700', 'type': 'C', 'cre...","{'value': 'autism, masturbation, hurricanes an...",False,False


In [17]:
# Get first post from each channel to get creation time
posts_sorted = p2.sort_values('time')
channel_created = posts_sorted.drop_duplicates(subset='channel', keep='first')
channel_created.head(5)

,user,channel,text,reply_count,reactions,replies,username,user_id,time
763054,NaN,annoucements,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-03-31 19:01:29.000200033
649965,NaN,3kp,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-03-31 19:01:30.000200033
151182,NaN,shitposting,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-03-31 21:05:13.000200033
327583,NaN,episodeguide,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-04-01 17:38:59.000200033
574950,NaN,culturewarz,<@UHH0R7TTP> has joined the channel,NaN,NaN,NaN,NaN,UHH0R7TTP,2019-04-03 03:20:56.000200033


In [18]:
# get channel and time created
chantime = channel_created[['channel', 'time']]
chantime = chantime.rename(columns={'time':'time_created'})
chantime

,channel,time_created
763054,annoucements,2019-03-31 19:01:29.000200033
649965,3kp,2019-03-31 19:01:30.000200033
151182,shitposting,2019-03-31 21:05:13.000200033
327583,episodeguide,2019-04-01 17:38:59.000200033
574950,culturewarz,2019-04-03 03:20:56.000200033
...,...,...
29834,conversation-corner,2024-01-16 06:01:28.298698902
622723,basketball,2024-01-24 16:27:28.651019096
361842,lotr,2024-02-19 14:39:11.854758978
510380,theater,2024-02-19 16:52:10.850178957


In [19]:
# merge to channels and get delta for 'channel age'
c2 = chans.copy()
c3 = pd.merge(c2, chantime, left_on='name', right_on='channel', how='left')
c3

,name,id,creator,members,purpose,pins,topic,is_archived,is_general,channel,time_created
0,bookstream,CHBC77PMX,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'discussions and channeling of thoug...,"[{'id': '1555531733.250500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,bookstream,2019-04-03 03:27:49.000200033
1,stackingsats,CHG467KSM,UHH0R7TTP,"[UH81VL1TK, UHEB1D3PA, UHH0R7TTP, UHH7GF9S9, U...","{'value': 'Stacking sats', 'creator': 'UHH0R7T...","[{'id': '1555032808.005500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,stackingsats,2019-04-07 16:39:42.000200033
2,annoucements,CHGKGAJJY,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'This channel is for workspace-wide ...,"[{'id': '1554065897.002200', 'type': 'C', 'cre...","{'value': 'Schedule and Info', 'creator': 'UHH...",False,True,annoucements,2019-03-31 19:01:29.000200033
3,shitposting,CHHPRTHGF,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1555442461.358700', 'type': 'C', 'cre...","{'value': 'autism, masturbation, hurricanes an...",False,False,shitposting,2019-03-31 21:05:13.000200033
4,3kp,CHJ3UTLMU,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'Updates and Timeline', 'creator': '...","[{'id': '1575338162.468200', 'type': 'C', 'cre...",{'value': 'Talk about the TV SHOW here. Fellas...,False,False,3kp,2019-03-31 19:01:30.000200033
...,...,...,...,...,...,...,...,...,...,...,...
231,hustle-craft,C06E77PFRLZ,UHH7GF9S9,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,hustle-craft,2024-01-14 18:41:04.062339067
232,basketball,C06FE7KMAMR,UHEB1D3PA,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH7GF9S9, U...",{'value': 'A channel where we talk about the s...,NaN,"{'value': 'Basketball', 'creator': 'UHEB1D3PA'...",False,False,basketball,2024-01-24 16:27:28.651019096
233,theater,C06K458MTFZ,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,theater,2024-02-19 16:52:10.850178957
234,lotr,C06KF15A2DB,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'beat you', 'creator': 'UHAC3TPCZ', ...","[{'id': '1708353745.957669', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,lotr,2024-02-19 14:39:11.854758978


In [20]:
now = datetime.now()
def calculate_age(past_date):
    delta = relativedelta(now, past_date)
    return f"{delta.years} years, {delta.months} months, {delta.days} days"

# Apply the function to create the 'age' column
c3['age'] = c3['time_created'].apply(calculate_age)
c3

,name,id,creator,members,purpose,pins,topic,is_archived,is_general,channel,time_created,age
0,bookstream,CHBC77PMX,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'discussions and channeling of thoug...,"[{'id': '1555531733.250500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,bookstream,2019-04-03 03:27:49.000200033,"4 years, 11 months, 26 days"
1,stackingsats,CHG467KSM,UHH0R7TTP,"[UH81VL1TK, UHEB1D3PA, UHH0R7TTP, UHH7GF9S9, U...","{'value': 'Stacking sats', 'creator': 'UHH0R7T...","[{'id': '1555032808.005500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,stackingsats,2019-04-07 16:39:42.000200033,"4 years, 11 months, 22 days"
2,annoucements,CHGKGAJJY,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'This channel is for workspace-wide ...,"[{'id': '1554065897.002200', 'type': 'C', 'cre...","{'value': 'Schedule and Info', 'creator': 'UHH...",False,True,annoucements,2019-03-31 19:01:29.000200033,"4 years, 11 months, 29 days"
3,shitposting,CHHPRTHGF,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1555442461.358700', 'type': 'C', 'cre...","{'value': 'autism, masturbation, hurricanes an...",False,False,shitposting,2019-03-31 21:05:13.000200033,"4 years, 11 months, 29 days"
4,3kp,CHJ3UTLMU,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'Updates and Timeline', 'creator': '...","[{'id': '1575338162.468200', 'type': 'C', 'cre...",{'value': 'Talk about the TV SHOW here. Fellas...,False,False,3kp,2019-03-31 19:01:30.000200033,"4 years, 11 months, 29 days"
...,...,...,...,...,...,...,...,...,...,...,...,...
231,hustle-craft,C06E77PFRLZ,UHH7GF9S9,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,hustle-craft,2024-01-14 18:41:04.062339067,"0 years, 2 months, 15 days"
232,basketball,C06FE7KMAMR,UHEB1D3PA,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH7GF9S9, U...",{'value': 'A channel where we talk about the s...,NaN,"{'value': 'Basketball', 'creator': 'UHEB1D3PA'...",False,False,basketball,2024-01-24 16:27:28.651019096,"0 years, 2 months, 5 days"
233,theater,C06K458MTFZ,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,theater,2024-02-19 16:52:10.850178957,"0 years, 1 months, 10 days"
234,lotr,C06KF15A2DB,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'beat you', 'creator': 'UHAC3TPCZ', ...","[{'id': '1708353745.957669', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,lotr,2024-02-19 14:39:11.854758978,"0 years, 1 months, 10 days"


In [22]:
# Get Count of Posts per Channel and Merge
p2.columns

Index(['user', 'channel', 'text', 'reply_count', 'reactions', 'replies',
       'username', 'user_id', 'time'],
      dtype='object')

In [25]:
postcount = p2.groupby(['channel', 'text']).size().reset_index(name='postcount')

postcount = postcount.groupby('channel')['postcount'].sum().reset_index()

postcount

,channel,postcount
0,3kp,30259
1,3kp-app,23
2,3kp-movieclub,10334
3,3kp-musikhaus,28
4,3kp-show-and-tell,156
...,...,...
235,wordleup,1160
236,work,5418
237,worldcup2022,495
238,ww-iii,3206


In [26]:
c4 = pd.merge(c3, postcount, left_on='name', right_on='channel', how='left')
c4

,name,id,creator,members,purpose,pins,topic,is_archived,is_general,channel_x,time_created,age,channel_y,postcount
0,bookstream,CHBC77PMX,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'discussions and channeling of thoug...,"[{'id': '1555531733.250500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,bookstream,2019-04-03 03:27:49.000200033,"4 years, 11 months, 26 days",bookstream,7875
1,stackingsats,CHG467KSM,UHH0R7TTP,"[UH81VL1TK, UHEB1D3PA, UHH0R7TTP, UHH7GF9S9, U...","{'value': 'Stacking sats', 'creator': 'UHH0R7T...","[{'id': '1555032808.005500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,stackingsats,2019-04-07 16:39:42.000200033,"4 years, 11 months, 22 days",stackingsats,31136
2,annoucements,CHGKGAJJY,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...",{'value': 'This channel is for workspace-wide ...,"[{'id': '1554065897.002200', 'type': 'C', 'cre...","{'value': 'Schedule and Info', 'creator': 'UHH...",False,True,annoucements,2019-03-31 19:01:29.000200033,"4 years, 11 months, 29 days",annoucements,43
3,shitposting,CHHPRTHGF,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1555442461.358700', 'type': 'C', 'cre...","{'value': 'autism, masturbation, hurricanes an...",False,False,shitposting,2019-03-31 21:05:13.000200033,"4 years, 11 months, 29 days",shitposting,50753
4,3kp,CHJ3UTLMU,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'Updates and Timeline', 'creator': '...","[{'id': '1575338162.468200', 'type': 'C', 'cre...",{'value': 'Talk about the TV SHOW here. Fellas...,False,False,3kp,2019-03-31 19:01:30.000200033,"4 years, 11 months, 29 days",3kp,30259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,hustle-craft,C06E77PFRLZ,UHH7GF9S9,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,hustle-craft,2024-01-14 18:41:04.062339067,"0 years, 2 months, 15 days",hustle-craft,45
232,basketball,C06FE7KMAMR,UHEB1D3PA,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH7GF9S9, U...",{'value': 'A channel where we talk about the s...,NaN,"{'value': 'Basketball', 'creator': 'UHEB1D3PA'...",False,False,basketball,2024-01-24 16:27:28.651019096,"0 years, 2 months, 5 days",basketball,11
233,theater,C06K458MTFZ,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,theater,2024-02-19 16:52:10.850178957,"0 years, 1 months, 10 days",theater,76
234,lotr,C06KF15A2DB,UHAC3TPCZ,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'beat you', 'creator': 'UHAC3TPCZ', ...","[{'id': '1708353745.957669', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,lotr,2024-02-19 14:39:11.854758978,"0 years, 1 months, 10 days",lotr,68


In [29]:
c4 = c4.sort_values(by='postcount', ascending=False)
c4

,name,id,creator,members,purpose,pins,topic,is_archived,is_general,channel_x,time_created,age,channel_y,postcount
6,culturewarz,CHMGSC22J,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'The front lines', 'creator': 'UHH0R...","[{'id': '1557329355.449700', 'type': 'C', 'cre...","{'value': 'Good vibes only', 'creator': 'UHH0R...",False,False,culturewarz,2019-04-03 03:20:56.000200033,"4 years, 11 months, 26 days",culturewarz,104384
3,shitposting,CHHPRTHGF,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': '', 'creator': '', 'last_set': 0}","[{'id': '1555442461.358700', 'type': 'C', 'cre...","{'value': 'autism, masturbation, hurricanes an...",False,False,shitposting,2019-03-31 21:05:13.000200033,"4 years, 11 months, 29 days",shitposting,50753
27,football,CLC13MDEW,UHH7GF9S9,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'Talk about whatever', 'creator': 'U...","[{'id': '1566054111.159800', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,football,2019-07-23 16:39:45.000200033,"4 years, 8 months, 6 days",football,43532
1,stackingsats,CHG467KSM,UHH0R7TTP,"[UH81VL1TK, UHEB1D3PA, UHH0R7TTP, UHH7GF9S9, U...","{'value': 'Stacking sats', 'creator': 'UHH0R7T...","[{'id': '1555032808.005500', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",False,False,stackingsats,2019-04-07 16:39:42.000200033,"4 years, 11 months, 22 days",stackingsats,31136
4,3kp,CHJ3UTLMU,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'Updates and Timeline', 'creator': '...","[{'id': '1575338162.468200', 'type': 'C', 'cre...",{'value': 'Talk about the TV SHOW here. Fellas...,False,False,3kp,2019-03-31 19:01:30.000200033,"4 years, 11 months, 29 days",3kp,30259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,the_third_one,C039ES32X8W,UHH0R7TTP,"[UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, UHHK3HG48, U...","{'value': 'What the hell goin on in here', 'cr...",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,the_third_one,2022-04-01 00:51:21.490849018,"1 years, 11 months, 28 days",the_third_one,8
195,eli5,C04UWAAAA3W,UHEB1D3PA,"[UH81VL1TK, UHAC3TPCZ, UHEB1D3PA, UHH0R7TTP, U...","{'value': 'For man/explaining', 'creator': 'UH...",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,eli5,2023-03-20 18:13:29.099029064,"1 years, 0 months, 9 days",eli5,8
202,puppyclipping,C056TTTSRSS,UHH0R7TTP,"[UH81VL1TK, UHAC3TPCZ, UHH0R7TTP]","{'value': '', 'creator': '', 'last_set': 0}",NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,puppyclipping,2023-05-10 00:35:38.470098972,"0 years, 10 months, 19 days",puppyclipping,7
226,normative_determinism,C06AQ4RSM18,UHH0R7TTP,"[UHH0R7TTP, UHK1N52Q6]",{'value': 'It's more fun to have it be blue th...,NaN,"{'value': '', 'creator': '', 'last_set': 0}",False,False,normative_determinism,2023-12-18 21:24:04.556509018,"0 years, 3 months, 11 days",normative_determinism,6


In [30]:
c4.columns

Index(['name', 'id', 'creator', 'members', 'purpose', 'pins', 'topic',
       'is_archived', 'is_general', 'channel_x', 'time_created', 'age',
       'channel_y', 'postcount'],
      dtype='object')

In [31]:
# Create posts per day

c4['post_per_day'] = c4['postcount'] / c4['age'].dt.days
c4

AttributeError: Can only use .dt accessor with datetimelike values

In [33]:
# Save temporarily
c4.to_csv('processed_json/channels.csv', index=False)